Finetune with MBart 


Tryout from preprocessing custom dataset for huggingface!!!!

In [61]:
from datasets import load_dataset, concatenate_datasets
import transformers
from transformers import MBartModel, MBartTokenizer, MBartConfig, pipeline, Trainer, TrainingArguments, MBartForConditionalGeneration, DataCollatorForSeq2Seq
import evaluate


Dataload using Huggingface 

In [22]:
train_data = load_dataset("json", data_files ="/home/sumire/main/discourse_context_mt/data/BSD-master/train.json")
dev_data = load_dataset("json", data_files ="/home/sumire/main/discourse_context_mt/data/BSD-master/dev.json")
test_data = load_dataset("json", data_files ="/home/sumire/main/discourse_context_mt/data/BSD-master/test.json")

Using custom data configuration default-0b58fbb4fdc24ac7


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/sumire/.cache/huggingface/datasets/json/default-0b58fbb4fdc24ac7/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

Using custom data configuration default-0b1429c816b88702


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/sumire/.cache/huggingface/datasets/json/default-0b1429c816b88702/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
train_data['dev']=dev_data['train']
train_data['test']=test_data['train']
data=train_data
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 670
    })
    dev: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 69
    })
    test: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 69
    })
})

In [42]:
for doc in data["train"]["conversation"]:
    for sent in doc:
        print (sent['en_sentence'])
        print (sent['ja_sentence'])

Hi this is the systems development department of Company K.
はい、K社システム開発部です。
My name is Takaichi from Company H.
H社の高市と申します。
Thank you as always.
いつもお世話になっております。
Thank you as always as well.
こちらこそ、お世話になっております。
Is Inada-san there?
稲田さんはいらっしゃいますか？
It seems I got a call from him around 1 hour ago.
1時間ほど前に、お電話いただいたそうなんですけど。
Yes, let me switch you over.
はい、代わります。
Please wait a moment.
少々お待ちください。
Inada-san, you have a call from Mr. Takaichi of Company H.
稲田さん、H社の高市様からお電話です。
Hello, this is Inada.
もしもし、稲田です。
This is Takaichi from Company H.
H社の高市です。
Thank you.
お世話様でーす。
Thank you as always.
お世話様でーす。
Thank you for the call earlier.
さっきはお電話ありがとうございました。
Sorry I was out.
外出していて、すみませんでした。
Not at all, I'm also sorry for calling when you were busy.
いいえ、こちらこそ、お忙しいところすみません。
We are going to Company A next Monday, right?
A社さんへ行くのって、来週月曜日でしたよね？
I thought we should maybe have a meeting once for that.
その打ち合わせ、一度したほうがいいと思いまして。
Right, let's do it.
そうですね、しましょう。
We have to prepare materials too so we should do it

In [40]:
inputs = [sent['en_sentence'] for doc in data["train"]["conversation"] for sent in doc]
targets = [sent['ja_sentence'] for doc in data["train"]["conversation"] for sent in doc]

inputs

['Hi this is the systems development department of Company K.',
 'My name is Takaichi from Company H.',
 'Thank you as always.',
 'Thank you as always as well.',
 'Is Inada-san there?',
 'It seems I got a call from him around 1 hour ago.',
 'Yes, let me switch you over.',
 'Please wait a moment.',
 'Inada-san, you have a call from Mr. Takaichi of Company H.',
 'Hello, this is Inada.',
 'This is Takaichi from Company H.',
 'Thank you.',
 'Thank you as always.',
 'Thank you for the call earlier.',
 'Sorry I was out.',
 "Not at all, I'm also sorry for calling when you were busy.",
 'We are going to Company A next Monday, right?',
 'I thought we should maybe have a meeting once for that.',
 "Right, let's do it.",
 'We have to prepare materials too so we should do it earlier.',
 'Takaichi-san, you said you are available anytime?',
 'Yes, when is good for you, Inada-san?',
 'If it is soon then...',
 'How about around ten thirty in the morning tomorrow?',
 'I will go to your place.',
 'Really

Preprocess Using Huggingface

In [52]:
model_checkpoint = "facebook/mbart-large-cc25"
configuration = MBartConfig()
tokenizer = MBartTokenizer.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="ro_RO")
model = MBartForConditionalGeneration(configuration).from_pretrained(model_checkpoint)

max_length = 128

def preprocess_function(data): # data should be splitted into train / dev / test internally
    inputs = [sent['en_sentence'] for doc in data["conversation"] for sent in doc]
    targets = [sent['ja_sentence'] for doc in data["conversation"] for sent in doc]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [56]:
tokenized_datasets = data.map(
    preprocess_function,
    batched=True,
    remove_columns=data["train"].column_names,
)

Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/json/default-0b58fbb4fdc24ac7/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-2a7f647e47751fba.arrow
Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/json/default-6b02b9f67a1bac72/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-de490fb6c1490c69.arrow
Loading cached processed dataset at /home/sumire/.cache/huggingface/datasets/json/default-0b1429c816b88702/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-c6b06cecb35ecb28.arrow


In [68]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
    dev: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2051
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2120
    })
})

DataCollator

In [63]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt") # not tf

Define Batch using Datacollator
###To Chryssa, decoder_input_ids now exists!####

In [64]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

# 'decoder_input_ids' is missing

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [65]:
batch["labels"]

tensor([[   572,   6808,    154,   1395,   2128,    610,  34606,   5182,     30,
              2, 250020],
        [     6,  38566, 160517,  14362,  55826,     30,      2, 250020,   -100,
           -100,   -100]])

In [66]:
batch["decoder_input_ids"]

tensor([[250020,    572,   6808,    154,   1395,   2128,    610,  34606,   5182,
             30,      2],
        [250020,      6,  38566, 160517,  14362,  55826,     30,      2, 250020,
              1,      1]])

In [ ]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[572, 6808, 154, 1395, 2128, 610, 34606, 5182, 30, 2, 250012]
[6, 38566, 160517, 14362, 55826, 30, 2, 250012]


Fine-tune using Trainer

In [69]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_datasets["train"],         # training dataset
    eval_dataset=tokenized_datasets["dev"],            # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer
)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 11.91 GiB total capacity; 1.29 GiB already allocated; 7.94 MiB free; 1.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [70]:
trainer.train()

NameError: name 'trainer' is not defined

In [75]:
# Training without using Trainer

inputs = tokenizer(inputs, text_target=targets, return_tensors="pt", padding=True)
print (inputs)

#model(**inputs)

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

BLEU metrics

In [ ]:
import evaluate 


In [ ]:
metric = evaluate.load("sacrebleu")